In [1]:
import pickle
import re
import numpy as np
import pandas as pd

In [2]:
fastText_vectors = {}
with open('fastText_vectors.txt', 'r') as f_handle:
    for line in f_handle:
        pieces = line[:-2].split(' ')
        fastText_vectors[pieces[0]] = np.array([float(fs) for fs in pieces[1:]])

In [3]:
lazaridou_vectors = {}
with open('vector_lazaridou.txt', 'r') as f_handle:
    for line in f_handle:
        line = re.sub('[\[\],]', '', line)
        # print(line)
        pieces = line[:-2].split(' ')
        # print(pieces)
        lazaridou_vectors[pieces[0]] = np.array([float(fs) for fs in pieces[1:]])

In [4]:
print(len(lazaridou_vectors))
print(len(fastText_vectors))
print(fastText_vectors['fawn'].shape)
print(lazaridou_vectors['fawn'].shape)



14656
14656
(200,)
(350,)


In [5]:
df_wordlist = pd.read_csv('wordList.csv', header=0, names=['Index', 'affix', 'derived', 'source'])

In [6]:
df_wordlist.shape

(8075, 4)

In [7]:
affix_list = list(df_wordlist['affix'].unique())
print(len(affix_list))
affix2index_dict = {}
for i in range(len(affix_list)):
    affix2index_dict[affix_list[i]] = i
# affix_list

54


In [8]:
def GetData(vector_model = fastText_vectors):    
    input_dim = vector_model['fawn'].shape[0]
    input_mat = np.zeros((df_wordlist.shape[0], input_dim + len(affix_list)))
    target_mat = np.zeros((df_wordlist.shape[0], input_dim))
    for index, row in df_wordlist.iterrows():
        input_mat[index, :input_dim] = vector_model[row.source]
        input_mat[index, input_dim + affix2index_dict[row.affix]] = 1
        target_mat[index, :] = vector_model[row.derived]
    return input_mat, target_mat, input_dim

In [9]:
input_mat

NameError: name 'input_mat' is not defined

### Neural Net Incoming

In [10]:
import tensorflow as tf
from sklearn.model_selection import KFold, cross_val_score

In [11]:
input_mat, target_mat, input_dim = GetData(lazaridou_vectors)
perm = np.random.permutation(input_mat.shape[0])
input_mat = input_mat[perm, :]
target_mat = target_mat[perm, :]

In [53]:
learning_rate = 0.001
training_epochs = 5
batch_size = 100

n_hidden_1 = input_dim + 27 + 50
n_input = input_dim + 54
n_output = input_dim

x = tf.placeholder('float', [None, n_input])
y = tf.placeholder('float', [None, n_output])

In [54]:
def mperceptron(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    # layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # layer_2 = tf.nn.relu(layer_2)
    
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

In [55]:
weights = {
    'h1' : tf.Variable(tf.random_normal([n_input, n_hidden_1], 0, 0.001)),
    # 'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], 0, 0.001)),
    'out' : tf.Variable(tf.random_normal([n_hidden_1, n_output], 0, 0.001))
}

biases = {
    'b1' : tf.Variable(tf.random_normal([n_hidden_1], 0, 0.001)),
    # 'b2': tf.Variable(tf.random_normal([n_hidden_2], 0, 0.001)),
    'out' : tf.Variable(tf.random_normal([n_output], 0, 0.001))
}

pred = mperceptron(x, weights, biases)

In [56]:
cost = tf.reduce_mean(tf.nn.l2_loss(pred - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf.global_variables_initializer()

In [59]:
sess = tf.Session()
k_fold = KFold(n_splits=5)
for train_indices, test_indices in k_fold.split(input_mat):
    # print('Train: %s | test: %s' % (train_indices, test_indices))
    train_x = input_mat[train_indices, :]
    train_y = target_mat[train_indices, :]
    test_x = input_mat[test_indices, :]
    test_y = target_mat[test_indices, :]
    
    sess.run(init)
    epochs = 5
    batches = 100
    batch_size = int(len(train_indices)/batches)
    for _ in range(epochs):
        for bx in range(batches):
            batch_indices = list(range((bx*batch_size), (bx + 1)*batch_size))
            _, c = sess.run([optimizer, cost], \
                            feed_dict={x: train_x[batch_indices, :],\
                                       y: train_y[batch_indices, :]})
    y_hat = sess.run(pred, feed_dict={x: test_x})
    
    # Check performance
    f_cossim = lambda x, y: np.sum(x*y)/(np.linalg.norm(x)*np.linalg.norm(y))
    cosSim_avg = 0
    for i in range(test_y.shape[0]):
        cosSim_avg += f_cossim(test_y[i, :], y_hat[i, :])
    cosSim_avg /= test_y.shape[0]
    print('Average cosine similarity: {}'.format(cosSim_avg))
sess.close()

Average cosine similarity: 0.3173085904743
Average cosine similarity: 0.32709070050371086
Average cosine similarity: 0.32558513878967665
Average cosine similarity: 0.3236761636585095
Average cosine similarity: 0.32766740124204935


In [ ]:
"""
FASTTEXT:
Avg. cosine similarity: 0.8 @
learning_rate = 0.008
epochs = 5
batches = 100
"""

"""
Lazaridou:
Avg. cosine similarity: 0.6 @
learning_rate = 0.008
epochs = 5
batches = 100
"""